<a href="https://colab.research.google.com/github/conglapgit45/Transformer_Basic/blob/main/BERT_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libs
!pip install -q -U transformers datasets accelerate evaluate

from datasets import load_dataset

ds = load_dataset('thainq107/ntc-scv')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/570 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.8M [00:00<?, ?B/s]

valid-00000-of-00001.parquet:   0%|          | 0.00/6.35M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/6.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [2]:
# Preprocessing

# Tokenization
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"    # bert-base-uncased

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
max_seq_length = 100
max_seq_length = min(max_seq_length, tokenizer.model_max_length)

def preprocess_function(examples):
    # Tokenize the texts
    result = tokenizer(
        examples["preprocessed_sentence"],
        padding = "max_length",
        max_length = max_seq_length,
        truncation = True
    )
    result["label"] = examples['label']
    return result

# Running the preprocessing pipeline on all the datasets
processed_dataset = ds.map(
    preprocess_function,
    batched = True,
    desc = "Running tokenizer on dataset",
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Running tokenizer on dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [6]:
# Modeling

from transformers import AutoConfig, AutoModelForSequenceClassification

num_labels = 2

config = AutoConfig.from_pretrained(
    model_name,
    num_labels = num_labels,
    finetuning_task = "text-classification"
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    config = config
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Metric

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    result = metric.compute(predictions=predictions, references=labels)
    return result

In [8]:
# Trainer
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "save_model",
    learning_rate = 2e-5,
    per_device_train_batch_size = 128,
    per_device_eval_batch_size = 128,
    num_train_epochs = 10,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = processed_dataset["train"],
    eval_dataset = processed_dataset["valid"],
    compute_metrics = compute_metrics,
    tokenizer = tokenizer,
)

trainer.train()

<ipython-input-8-610e601b3698>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.446591,0.806300
2,No log,0.484516,0.791900
3,0.455400,0.405385,0.821900
4,0.455400,0.391195,0.833800
5,0.350600,0.405101,0.837200
6,0.350600,0.389068,0.840100
7,0.291300,0.392797,0.840100
8,0.291300,0.413715,0.841400
9,0.248000,0.432103,0.842700
10,0.248000,0.431301,0.840700


TrainOutput(global_step=2350, training_loss=0.31940361834586933, metrics={'train_runtime': 3008.7255, 'train_samples_per_second': 99.71, 'train_steps_per_second': 0.781, 'total_flos': 7761761640000000.0, 'train_loss': 0.31940361834586933, 'epoch': 10.0})

In [16]:
# Test with user's input

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

model_path = "save_model/checkpoint-2350"  # Update with the actual path to your last checkpoint directory
classifier = pipeline(
    "text-classification",
    model=AutoModelForSequenceClassification.from_pretrained(model_path),
    tokenizer=AutoTokenizer.from_pretrained(model_path)
)

# Get user input
user_input = input("Enter a sentence: ")

# Classify the input
result = classifier(user_input)

# Print the result
print(result)
print("LABEL_0 is Negative")
print("LABEL_1 is Positive")

Device set to use cuda:0


Enter a sentence: Trang trí không đẹp
[{'label': 'LABEL_0', 'score': 0.9613540172576904}]
LABEL_0 is Negative
LABEL_1 is Positive
